In [ ]:
# !pip install sumolib

In [ ]:
# pip install --upgrade python-igraph folium==0.12.1.post1 geojson==2.5.0 geopandas==0.10.2 pandas==1.1.5 statsmodels==0.13.0


In [ ]:
# !pip install pandas==1.2.5

In [2]:
import sumolib
import pandas as pd
import geopandas as gpd
from shapely import wkt
import osmnx as ox
import subprocess
import folium
import openrouteservice as ors


In [4]:
net = sumolib.net.readNet('osm.net.xml.gz')

In [5]:
edge_list = net.getEdges()
node_list = net.getNodes()

print(f"The road network has {len(edge_list)} edges and {len(node_list)} nodes.")

The road network has 55974 edges and 25171 nodes.


In [7]:

outflow_df = pd.read_csv('outflow_per_region.csv')
outflow_df['zone_geometry'] = outflow_df['zone_geometry'].apply(wkt.loads)
outflow_gdf = gpd.GeoDataFrame(outflow_df, geometry='zone_geometry')

In [9]:
# cerco i centroidi della regione per cui outflow è max e centroidi di una regione centrale a caso ( 'LEUVEN-CENTRUM' in questo caso )
geom_max_out = outflow_gdf.loc[outflow_gdf['flows_car_aggr'].idxmax(), 'zone_geometry'].centroid
geom_centro = outflow_gdf.loc[outflow_gdf['SECNAAM'] == 'LEUVEN-CENTRUM', 'zone_geometry'].iloc[0].centroid
lat_centro, lon_centro = geom_max_out.y, geom_max_out.x
lat_maxflow, lon_maxflow = geom_centro.y, geom_centro.x

# convert into SUMO coordinates (x, y)
x_maxflow, y_maxflow = net.convertLonLat2XY(lon_maxflow, lat_maxflow)
x_centro, y_centro = net.convertLonLat2XY(lon_centro, lat_centro)

print(f"SUMO reference coordinates (x,y) for centrum: {x_centro,y_centro}")
print(f"SUMO reference coordinates (x,y) for maxflow: {x_maxflow,y_maxflow}")

SUMO reference coordinates (x,y) for centrum: (13118.32514550921, 11214.115449559875)
SUMO reference coordinates (x,y) for maxflow: (12883.954596722615, 12184.558083221316)


In [10]:
# the set of Neighboring edges
candiates_edges = net.getNeighboringEdges(x_maxflow, y_maxflow, r=25)
edges_and_dist_maxflow = sorted(candiates_edges, key = lambda x: x[1])

candiates_edges_centro = net.getNeighboringEdges(x_centro, y_centro, r=25)
edges_and_dist_centro = sorted(candiates_edges_centro, key = lambda x: x[1])


In [11]:
x_maxflow,y_maxflow

(12883.954596722615, 12184.558083221316)

In [14]:
closest_edge_maxflow = edges_and_dist_maxflow[0][0]
closest_edge_centro = edges_and_dist_centro[0][0]

print(f"Name closest edge to maxflow: {closest_edge_maxflow.getName()}")
print(f"Edge ID: {closest_edge_maxflow.getID()}")

print(f"Name closest edge to centrum: {closest_edge_centro.getName()}")
print(f"Edge ID: {closest_edge_centro.getID()}")

Name closest edge to maxflow: 
Edge ID: 385589764#2
Name closest edge to centrum: Paul Lebrunstraat
Edge ID: -3351197


In [15]:
closest_edge_maxflow, closest_edge_centro

(<edge id="385589764#2" from="1573474888" to="3663114400"/>,
 <edge id="-3351197" from="16483865" to="16526534"/>)

In [16]:
e_origin, e_destination = closest_edge_maxflow, closest_edge_centro

# shortest path 
shortest_path = net.getOptimalPath(e_origin, e_destination, fastest=False)
shortest_path
# print(f"Number of edges in the path: {len(shortest_path[0])}")
# print(f"Cost [m]: {shortest_path[1]}")


(None, inf)

In [19]:
rete = r"C:\Users\Valeria\Documents\Università\DS\GEO\osm.net.xml\osm.net.xml"
G = ox.graph_from_xml(rete, simplify=True)

ox.plot_graph(G, edge_color='black', node_color='red', bgcolor='white')

InsufficientResponseError: No data elements in server response. Check query location/filters and log.

## duarouter - 
non ho ben capito come funziona

In [12]:

command_str_astar = "duarouter --route-files traffic_demand_leuven.rou.xml "+\
        " --net-file osm.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter_astar.rou.xml --routing-algorithm astar"

p = subprocess.Popen(command_str_astar, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

In [14]:
command_str_w = "duarouter --route-files traffic_demand_leuven.rou.xml "+\
        " --net-file osm.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter_astar.rou.xml --routing-algorithm astar --weights.random-factor 300"

p = subprocess.Popen(command_str_w, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

### OpenStreetMap routing API 


In [21]:
YOUR_KEY = '5b3ce3597851110001cf624899fee7dd897b42e19b4300590818493a'

In [1]:
import openrouteservice

In [26]:

x_origin, y_origin = e_origin.getFromNode().getCoord()
lon_origin, lat_origin = net.convertXY2LonLat(x_origin, y_origin)


x_dest, y_dest = e_destination.getFromNode().getCoord()
lon_dest, lat_dest = net.convertXY2LonLat(x_dest, y_dest)

coordinates = [[lon_origin, lat_origin], [lon_dest, lat_dest]]
client = ors.Client(key=YOUR_KEY)


route_osm = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        preference="shortest",
        format='geojson',
        geometry_simplify=False,
        validate=True)

# extract the GPS points (lon, lat) of the fastest route
gps_points_osm = [coord for coord in route_osm['features'][0]['geometry']['coordinates']]

In [30]:

m = folium.Map(location=[lat_origin, lon_origin], tiles='cartodbpositron', zoom_start=13)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in gps_points_osm], popup = 'shortest').add_to(m)
m

In [31]:
x_origin, y_origin = e_origin.getFromNode().getCoord()
lon_origin, lat_origin = net.convertXY2LonLat(x_origin, y_origin)


x_dest, y_dest = e_destination.getFromNode().getCoord()
lon_dest, lat_dest = net.convertXY2LonLat(x_dest, y_dest)

coordinates = [[lon_origin, lat_origin], [lon_dest, lat_dest]]
client = ors.Client(key=YOUR_KEY)


route_osm = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        preference="recommended",
        format='geojson',
        geometry_simplify=False,
        validate=True)

# extract the GPS points (lon, lat) of the fastest route
gps_points_osm_recom = [coord for coord in route_osm['features'][0]['geometry']['coordinates']]


folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in gps_points_osm_recom], color='red', popup='recommended').add_to(m)
m